In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import datetime as dt
from tqdm import tqdm
import matplotlib.pyplot as plt

'''#import plotly for interactive chart
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='richwolff', api_key='v0qPC120X33yPvAMDQXi')
from plotly.graph_objs import * '''

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

from collections import defaultdict
%matplotlib inline

<h2>Load books into dataframe</h2>

In [2]:
d = defaultdict(list)

strtofind = r'";"'
with open('../data/raw/BX-Books.csv','r',encoding='8859') as file:
    for i,line in enumerate(file):
        d[i] = re.sub(strtofind,'||',line.replace('&amp;','&')).replace('"','').replace('\n','').split('||')
        
books_df = pd.DataFrame(data=list(d.values())[1:],index=list(d.keys())[1:],columns=d[0])
books_df.head()

del d

<H2> Load Users Into DF </H2>

In [3]:
## Load users file and display first 5 rows
users_df = pd.read_csv('../data/raw/BX-Users.csv',sep=';',encoding='8859')
users_df.tail(5)

,User-ID,Location,Age
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN
278857,278858,"dublin, n/a, ireland",NaN


<h2>Load User Ratings Of Books</h2>

In [4]:
ratings_df = pd.read_csv('../data/raw/BX-Book-Ratings.csv',sep=';',encoding='8859',dtype={'Book-Rating':np.int}).sort_values('User-ID')
ratings_df.head()

,User-ID,ISBN,Book-Rating
9561,2,0195153448,0
9562,7,034542252,0
9572,8,0771025661,0
9580,8,1881320189,7
9579,8,1575663937,6


In [5]:
## Join user and book data to ratings data
ratings = ratings_df.set_index('User-ID').join(users_df.set_index('User-ID')).reset_index().set_index('ISBN').join(books_df.set_index('ISBN'))

## Split out users from the USA
us_ratings = ratings[(ratings['Location'].str.lower().str.contains('usa')) | (ratings['Location'].str.lower().str.contains('states'))].reset_index()
us_ratings.head()

,ISBN,User-ID,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0375404120,266865,0,"reston, virginia, usa",33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,)440206529,238681,0,"milford, ohio, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,)452273056,111422,8,"avon, massachusetts, usa",59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,*0515128325,190925,0,"hobe sound, florida, usa",51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/8741060773,52796,9,"sumner, iowa, usa",0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h2>Drop books that have only been read once. These can not be used for good recommendations</h2>

In [13]:
# How many times a book must have been read to keep it in ratings
min_book_read_count = 2

isbn_val_counts = us_ratings['ISBN'].value_counts()
books_to_keep = set(isbn_val_counts[isbn_val_counts>=min_book_read_count].index)

filtered_us_ratings = us_ratings[us_ratings['ISBN'].isin(books_to_keep)]

<h2>Split data into training and test sets</h2>

In [16]:
ratings_us_training,ratings_us_test = train_test_split(filtered_us_ratings,test_size=.20,random_state=21)

<h2>Insert data into graph data structure</h2>

In [18]:
import datetime as dt
user_nodes = set(ratings_us_training['User-ID'].unique())
book_nodes = set(ratings_us_training['ISBN'].unique())

# Build the graph structure from pandas data frame
G = nx.from_pandas_edgelist(ratings_us_training,'User-ID','ISBN',['Book-Rating'])

# Compute DCS
dcs = nx.bipartite.degree_centrality(G,user_nodes)

# Add Meta Data
for i,row in ratings_us_training.iterrows():
    user_node = G.node[row['User-ID']]
    book_node = G.node[row['ISBN']]
    
    user_node['Age'] = row['Age']
    user_node['Location'] = row['Location']
    user_node['bipartite'] = 'user'
    user_node['dcs'] = dcs[row['User-ID']]
    
    
    book_node['bipartite'] = 'book'
    book_node['Book-Title'] = row['Book-Title']
    book_node['Book-Author'] = row['Book-Author']
    book_node['Publisher'] = row['Publisher']
    book_node['Publication_Year'] = row['Year-Of-Publication']
    book_node['dcs'] = dcs[row['ISBN']]

<h2>Create a user and book biadjacency matrix with users as rows and books as columns </h2>

In [250]:
#using the rating as weight to add weight to explicit reviews to similarity scores
user_arr = np.array(list(user_nodes))
books_arr = np.array(list(book_nodes))
user_adj_matrix = nx.bipartite.biadjacency_matrix(G,row_order=user_nodes,column_order=book_nodes,weight='Book-Rating')

<h2>Create a user x user matrix with the cosine similarities as their intersection value</h2>

In [251]:
# Take cosine similarities of users based on ratings they've given each book (column)
user_sims = cosine_similarity(user_adj_matrix,dense_output=False)
user_sims.setdiag(0)
user_sims_coo = user_sims.tocoo()

C:\Users\rich.wolff\AppData\Local\Continuum\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [252]:
def collaborative_filter(selected_user,node_list,user_matrix,top_n_similarities):
    
    def node_similiarities(node, node_list, matrix):
        '''Creates a numpy array of node similiarities (user or books)'''
        indices = np.where(matrix.row == np.where(node_list==node)[0])[0]
        matrix_sims_node = []
        matrix_sims_score = []
        nodes_sim = defaultdict(list)
        for idx in indices:
            cos_sim = (matrix.data[idx])
            nodes_sim[cos_sim].append(node_list[matrix.col[idx]])
        return nodes_sim
    
    def user_neighbor_books(selected_user, user_similarity_dict,top_n_similarities):
        '''accepts a 2d array with users in the first column and similarities in the 2nd
           returns top 10 books with scores'''
        books = defaultdict(lambda: defaultdict(float))
        for key in sorted(user_similarity_dict.keys(),reverse=True)[:top_n_similarities]:
            for usr_lookup in user_similarity_dict[key]:
                for bk in set(G.neighbors(usr_lookup)).difference(G.neighbors(selected_user)):
                    book_rating = G[usr_lookup][bk]['Book-Rating']
                    books[bk]['count'] += 1
                    books[bk]['cosine'] += key
                    books[bk]['rating'] += book_rating
                    books[bk]['implicit_ratings'] += 1 if book_rating == 0 else 0
                    books[bk]['explicit_ratings'] += 1 if book_rating > 0 else 0
                    books[bk]['avg_cosine'] = books[bk]['cosine']/books[bk]['count']  
                    books[bk]['avg_rating'] = books[bk]['rating']/books[bk]['count']
                    if books[bk]['explicit_ratings'] > 0:
                        books[bk]['avg_explicit_rating'] = books[bk]['rating']/books[bk]['explicit_ratings']
                    if books[bk]['implicit_ratings'] > 0:
                        books[bk]['explicit_implicit_ratio'] = books[bk]['explicit_ratings']/books[bk]['implicit_ratings']
        
        return books
    
    def books_dict_to_df(books_list):
        books_list = [(b,
               d['avg_rating'],
               d['avg_explicit_rating'],        
               d['avg_cosine'],
               d['explicit_implicit_ratio'],
               d['count'],
               d['cosine'],
               d['rating'],
               d['implicit_ratings']) for b,d in zip(user_books_df.keys(),user_books_df.values())]
        df_columns = ['ISBN','avg_rating','avg_explicit_rating','avg_cosine','explicit_implicit_ratio','user_count','cosines','ratings','implicit_ratings']
        ret_df = pd.DataFrame(books_list,columns=df_columns)
        ret_df['user'] = selected_user
        return ret_df


    user_sims_nodes = node_similiarities(selected_user,user_arr,user_sims_coo)
    
    user_books_df = user_neighbor_books(selected_user=selected_user,
                                        user_similarity_dict=user_sims_nodes,
                                        top_n_similarities=top_n_similarities)
    
    return books_dict_to_df(user_books_df).set_index(['user','ISBN'])

<h2>Test 1 user</h2>

In [253]:
selected_user = 35859     

n_recommendations = 10
recommended_books = collaborative_filter(selected_user,user_arr,user_sims_coo,100)
recommended_books = recommended_books.sort_values(['user_count','explicit_implicit_ratio'],ascending=False)
recommended_books = recommended_books.head(n_recommendations)

# Pull actual purchases
sel_user_actual = ratings_us_test[ratings_us_test['User-ID']==selected_user]['ISBN'].to_frame()
sel_user_actual['read'] = 1
sel_user_actual['user'] = selected_user
sel_user_actual.set_index(['user','ISBN'],inplace=True)

# TOP N Recommendation based on cosines
recommended_books.join(sel_user_actual).fillna(0)

avg_rating  avg_explicit_rating  avg_cosine  \
user  ISBN                                                      
35859 0312966970    7.375000             8.428571    0.058533   
      0312980140    6.800000             8.500000    0.054018   
      0446610038    2.000000            10.000000    0.042775   
      0312976275    8.400000             8.400000    0.057504   
      0439064872    4.500000             9.000000    0.047471   
      0316569321    2.000000             8.000000    0.046403   
      0345465083    1.500000             6.000000    0.043949   
      0609804138    5.666667             8.500000    0.053363   
      0140293248    4.333333             6.500000    0.056952   
      0142001740    5.666667             8.500000    0.047589   

                  explicit_implicit_ratio  user_count   cosines  ratings  \
user  ISBN                                                                 
35859 0312966970                 7.000000         8.0  0.468267     59.0   
      0312980140                 4.000000         5.0  0.270089     34.0   
      0446610038                 0.250000         5.0  0.213873     10.0   
      0312976275                 0.000000         5.0  0.287520     42.0   
      0439064872                 1.000000         4.0  0.189884     18.0   
      0316569321                 0.333333         4.0  0.185612      8.0   
      0345465083                 0.333333         4.0  0.175798      6.0   
      0609804138                 2.000000         3.0  0.160090     17.0   
      0140293248                 2.000000         3.0  0.170856     13.0   
      0142001740                 2.000000         3.0  0.142766     17.0   

                  implicit_ratings  read  
user  ISBN                                
35859 0312966970               1.0   0.0  
      0312980140               1.0   1.0  
      0446610038               4.0   1.0  
      0312976275               0.0   0.0  
      0439064872               2.0   1.0  
      0316569321               3.0   0.0  
      0345465083               3.0   0.0  
      0609804138               1.0   1.0  
      0140293248               1.0   0.0  
      0142001740               1.0   1.0

<h2>Bring back scores for multiple users in test file</h2>

In [254]:
top_n_cosines = 25 
n_recommendations = 50
test_ratings_user_count = 50

test_ratings_user_valcount = ratings_us_test['User-ID'].value_counts()
filtered_test_ratings = test_ratings_user_valcount[test_ratings_user_valcount>test_ratings_user_count]
users_to_test = filtered_test_ratings.index.values

metrics = np.empty(len(users_to_test))

group_df = []

for i,sel_user in enumerate(users_to_test):
    recommended_books = collaborative_filter(sel_user,user_arr,user_sims_coo,top_n_cosines)
    
    sel_user_actual = ratings_us_test[ratings_us_test['User-ID']==sel_user]['ISBN'].to_frame()
    sel_user_actual['read'] = 1
    sel_user_actual['user'] = sel_user
    sel_user_actual.set_index(['user','ISBN'],inplace=True)
    
    group_df.append(recommended_books.sort_values(['user_count','avg_explicit_rating'],ascending=False).head(n_recommendations).join(sel_user_actual))
group_df = pd.concat(group_df).fillna(0)    

In [255]:
group_df.head(20)
X = group_df.drop(['read'],axis=1)
y = group_df['read']

<h2>Import sklearn items</h2>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, auc

<h2>Logistic Regression</h2>

In [280]:
# Log reg model
lr = LogisticRegression()
lr.fit(X,y)

log_reg_coefs = pd.DataFrame(lr.coef_,columns=X.columns).T.sort_values(0,ascending=False)

confusion = confusion_matrix(y,lr.predict(X))
true_notread, false_notread, false_read, true_read = confusion.ravel()
pd.crosstab(y,lr.predict(X), rownames=['True'], colnames=['Predicted'],)

Predicted,0.0,1.0
True,,
0.0,20019,30
1.0,767,34


In [281]:
print('Percent Predicted Read Correctly: {:.4f}'.format(true_read/(true_read+false_read)))
print('Percent Predicted Not Read Correctly: {:.4f}'.format(true_notread/(true_notread+false_notread)))

Percent Predicted Read Correctly: 0.0424
Percent Predicted Not Read Correctly: 0.9985


In [278]:
#RANDOM FOREST CLASSIFIER

rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X,y)

pd.DataFrame(rfc.feature_importances_,X.columns,['Feature Importances']).sort_values('Feature Importances',ascending=False)

ypred = rfc.predict(X)
confusion = confusion_matrix(y,ypred)
true_notread, false_notread, false_read, true_read = confusion.ravel()

print('Random Forests Classifier Confusion Matrix')
pd.crosstab(y,ypred, rownames=['True'], colnames=['Predicted'],)

Random Forests Classifier Confusion Matrix


Predicted,0.0,1.0
True,,
0.0,20030,19
1.0,264,537


In [276]:
print('Percent Predicted Read Correctly: {:.4f}'.format(true_read/(true_read+false_read)))
print('Percent Predicted Not Read Correctly: {:.4f}'.format(true_notread/(true_notread+false_notread)))

Percent Predicted Read Correctly: 0.6629
Percent Predicted Not Read Correctly: 0.9992


In [259]:
cross_val_score(rfc,X,y,cv=10,)

array([0.83029722, 0.83741007, 0.87002398, 0.85083933, 0.89976019,
       0.88872902, 0.92470024, 0.90839329, 0.93860911, 0.94193858])

In [260]:
params = {'n_estimators':np.linspace(10,50,2,dtype=np.int),'criterion':['gini','entropy'],'n_jobs':[1,2,3]}
gscv = GridSearchCV(rfc,params,cv=5)
gscv.fit(X,y)
print(gscv.best_params_)
print(gscv.best_score_)

{'criterion': 'entropy', 'n_estimators': 10, 'n_jobs': 1}
0.908441247002398


In [194]:
params = {'n_estimators':np.linspace(1,50,2,dtype=np.int),'criterion':['gini','entropy'],'n_jobs':[1,2,3]}
gscv = GridSearchCV(rfc,params,cv=5)
gscv.fit(X,y)
print(gscv.best_params_)
print(gscv.best_score_)

{'criterion': 'entropy', 'n_estimators': 50, 'n_jobs': 1}
0.7954976303317536


In [195]:
params = {'n_estimators':np.linspace(1,50,2,dtype=np.int),'criterion':['gini','entropy'],'n_jobs':[1,2,3]}
gscv = GridSearchCV(rfc,params,cv=5)
gscv.fit(X,y)
print(gscv.best_params_)
print(gscv.best_score_)

{'criterion': 'entropy', 'n_estimators': 50, 'n_jobs': 1}
0.795260663507109


In [196]:
params = {'n_estimators':np.linspace(25,75,2,dtype=np.int),'criterion':['gini','entropy'],'n_jobs':[1,2,3]}
gscv = GridSearchCV(rfc,params,cv=5)
gscv.fit(X,y)
print(gscv.best_params_)
print(gscv.best_score_)

{'criterion': 'gini', 'n_estimators': 75, 'n_jobs': 2}
0.7893364928909953


In [197]:
params = {'n_estimators':np.linspace(25,75,2,dtype=np.int),'criterion':['gini','entropy'],'n_jobs':[1,2,3]}
gscv = GridSearchCV(rfc,params,cv=5)
gscv.fit(X,y)
print(gscv.best_params_)
print(gscv.best_score_)

{'criterion': 'entropy', 'n_estimators': 75, 'n_jobs': 1}
0.790521327014218


x = sorted(metrics)
n = len(x)
y = np.arange(1, n+1)/n

plt.plot(x,y);
plt.title('Median: {}'.format(np.median(metrics)));

<H1>DONT DELETE AFTER THIS. FOR BOOK SIMILARITIES</H1>

<h2>Create a book x book matrix with cosine similarities as their intersection values</h2>

In [ ]:
book_adj_matrix = user_adj_matrix.T
book_sims = cosine_similarity(book_adj_matrix,dense_output=False)
book_sims.setdiag(0)
book_sims_coo = book_sims.tocoo()

In [ ]:
def bk_mtx_ind(coo_mtx):
    '''Loop through coordinate matrix rows and store the idx location of values in a dictionary
       dict[row].append(idx) 
       
       This will allow for fast lookups later vs looping through a 300MM list thousands of times later
       
       Move complexity from O(n^2) to O(N) (iterate matrix rows once, N every lookup is O(1) in python dicts)
    '''  
    bk_mtx_lkup = defaultdict(list)
    
    for i,bk in enumerate(coo_mtx.row):
        bk_mtx_lkup[bk].append(i)
        
    return bk_mtx_lkup

b = dt.datetime.now()
bk_mtx_lkup = bk_mtx_ind(coo_mtx=book_sims_coo)
print(dt.datetime.now() - b)

In [83]:
def book_collab_filter(selected_user,node_list,book_matrix,book_matrix_lookup,top_n_similarities):
    
    def node_similiarities(nodes, node_list, matrix,book_matrix_lookup): #### O(n^2), can i reduce this?
        '''Creates a numpy array of node similiarities (user or books)'''
        nodes_sim = defaultdict(list)
        
        # find the positions of all the books read by the user
        book_positions = np.where(np.isin(node_list,nodes))[0]
        
        #For each data point from node, append value book to dictionary key cosine similarity
        for i,pos in enumerate(book_positions):
            for idx in book_matrix_lookup[pos]:
                bk = node_list[matrix.col[idx]]
                if not bk in nodes:
                    nodes_sim[matrix.data[idx]].append(bk)
                
        return nodes_sim
    
    def user_neighbor_books(book_similarity_dict, top_n_similarities,user_read_books):
        '''accepts a 2d array with users in the first column and similarities in the 2nd
           returns top 10 books with scores'''
        books = defaultdict(lambda: defaultdict(float))
        for key in sorted(book_similarity_dict.keys(),reverse=True)[:top_n_similarities]:
            for bk in book_similarity_dict[key]:
                if not bk in user_read_books:
                    books[bk]['count'] += 1
                    books[bk]['cosine'] += key
                    books[bk]['avg_cosine'] = books[bk]['cosine']/books[bk]['count']  
                    books[bk]['avg_rating'] = books[bk]['rating']/books[bk]['count']
        return books
    
    def books_dict_to_df(books_list):
        books_list = [(b,
               d['avg_rating'],        
               d['avg_cosine'],
               d['count'],
               d['cosine']) for b,d in zip(book_books_df.keys(),book_books_df.values())]
        df_columns = ['book','avg_rating','avg_cosine','book_count','cosines']
        return pd.DataFrame(books_list,columns=df_columns)
    
    ## return top cosine similarities books based on books user read
    
    ## For books user read
        # Get books with high cosine similarities to each book
        # count and store in dict/data frame

    ### GET BOOK LIST
    dfa = defaultdict(list)
    user_read_books = set(G.neighbors(selected_user))
    for x in list(user_read_books):
        dfa[G[selected_user][x]['Book-Rating']].append(x)  
        
    top_5_keys = sorted(dfa.keys())[-1::]
    top_5_rated_books = []
    for x in top_5_keys:
        top_5_rated_books += dfa[x]
    
    
    book_sims_nodes = node_similiarities(np.array(top_5_rated_books),node_list,book_matrix,book_matrix_lookup)
    
    book_books_df = user_neighbor_books(book_similarity_dict=book_sims_nodes,
                                        top_n_similarities=top_n_similarities,user_read_books=user_read_books)
    
    return books_dict_to_df(book_books_df).set_index('book')

In [84]:
len(list(G.neighbors(selected_user)))

3896

In [94]:
selected_user = 153662
res = book_collab_filter(selected_user,books_arr,book_sims_coo,bk_mtx_lkup,20)

In [95]:
# Pull actual purchases
sel_user_actual = ratings_us_test[ratings_us_test['User-ID']==selected_user]['ISBN'].to_frame()
sel_user_actual['read'] = 1

In [98]:
# TOP N Recommendation based on cosines
n_recommendations = 15
reco_test = res.sort_values(['book_count','avg_cosine'],ascending=False).head(n_recommendations).join(sel_user_actual.set_index('ISBN'))
reco_test

,avg_rating,avg_cosine,book_count,cosines,read
book,,,,,
0505524236,0.0,0.724476,2.0,1.448952,NaN
0671737694,0.0,0.707107,2.0,1.414214,NaN
0874062853,0.0,0.707107,2.0,1.414214,NaN
0671535307,0.0,0.707107,2.0,1.414214,NaN
8440629583,0.0,0.707107,2.0,1.414214,NaN
0831713097,0.0,0.707107,2.0,1.414214,NaN
0590414151,0.0,0.707107,2.0,1.414214,NaN
1569870454,0.0,0.707107,2.0,1.414214,NaN
1570825181,0.0,0.707107,2.0,1.414214,NaN


False

In [ ]:
top_n_cosines = 50 
n_recommendations = 15
test_ratings_user_count = 50

test_ratings_user_valcount = ratings_us_test['User-ID'].value_counts()
filtered_test_ratings = test_ratings_user_valcount[test_ratings_user_valcount>test_ratings_user_count]
users_to_test = filtered_test_ratings.index.values

metrics = np.empty(len(users_to_test))

for i,sel_user in enumerate(users_to_test):
    recommended_books = collaborative_filter(sel_user,user_arr,user_sims_coo,top_n_cosines)
    sel_user_actual = ratings_us_test[ratings_us_test['User-ID']==sel_user]['ISBN'].to_frame()
    sel_user_actual['read'] = 1
    reco_test = recommended_books.sort_values(['cosines','avg_rating'],ascending=False).head(n_recommendations).join(sel_user_actual.set_index('ISBN'))
    metrics[i] = np.sum(reco_test['read'])/n_recommendations

In [ ]:
def user_book_cosines(usr):
    read_books = list(G[usr])
    books = defaultdict(lambda: defaultdict(float))
    for bk in read_books:
        book_nodes,book_scores = node_similiarities(bk,books_arr,book_sims_coo)
        for book,score in zip(book_nodes,book_scores):
            books[book]['cosine_total'] +=score
            books[book]['count'] += 1
            books[book]['avg_cosine'] = books[book]['cosine_total']/books[book]['count']
        
    return books

book_list = user_book_cosines(selected_user)
book_list = [(b,
               d['cosine_total'],
               d['count'],
               d['avg_cosine']) for b,d in zip(book_list.keys(),book_list.values())]
book_book_df = pd.DataFrame(book_list,columns=['book','book_book_cosine','book_book_count','book_book_avgcosine'])
book_book_df.sort_values('book_book_count',ascending=False,inplace=True)
book_book_df.set_index('book',inplace=True)


In [ ]:
list(G[selected_user])[0]

In [ ]:
list(G.neighbors('034542252'))

In [ ]:
len(list(G.neighbors(166409)))

In [ ]:
user_scores[user_scores>np.percentile(user_scores,1)]

In [ ]:
book_reco

<h2>Build books into dataframe</h2>

In [ ]:
book_book_df.head()

In [ ]:
tst = book_reco.join(book_book_df,how='outer')

np.percentile(tst['avg_user_cosines'].fillna(0),99.9) #book_book_avgcosine, #avg_user_cosines

In [ ]:
sel_user_actual = ratings_us_test[ratings_us_test['User-ID']==selected_user]['ISBN'].to_frame()
sel_user_actual['read'] = 1
tst1 = tst.join(sel_user_actual.set_index('ISBN'),how='outer')
np.sum(tst1['read']>0)/len(tst1)

In [ ]:

tst1['book_book_avgcosine'].hist()

In [ ]:
tst1['avg_user_cosines'].hist()

In [ ]:
ratings_df['User-ID'].value_counts()[ratings_df['User-ID'].value_counts()<3].sum()/len(ratings_df['User-ID'].unique())

In [ ]:
ratings_us_test['User-ID'].unique()

In [ ]:
book_reco_filtered = book_reco[book_reco['user_count'] >= 10]
book_reco_filtered['avg_user_cosine'] = book_reco_filtered['user_cosines']/book_reco_filtered['user_count']
top_10_books = set(book_reco_filtered.sort_values('avg_user_cosine',ascending=False).iloc[:10]['book'])
top_10_books.intersection(set(ratings_us_test[ratings_us_test['User-ID']==selected_user].sort_values('ISBN')['ISBN']))
#top_10_books